In [3]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

from langchain_openai import ChatOpenAI
with open('openai_api_key.txt', 'r') as f:
    key = f.read()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/32

In [4]:
LLM = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.2, api_key=key)
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document

In [22]:
LLM.invoke('Hi. test connection. who are you?')

AIMessage(content="Hello! I'm an AI language model created by OpenAI. I'm here to assist you with a wide range of topics, answer questions, and provide information. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 16, 'total_tokens': 54, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae1d2e42-29c1-4e09-93da-b71fac83e18e-0', usage_metadata={'input_tokens': 16, 'output_tokens': 38, 'total_tokens': 54, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [46]:
import random
import time

# Constants
GRID_SIZE = 20
NUM_MONSTERS = 6
NUM_SWAMPS = 5
CABINET_LOCATIONS = 3
CLUE_COUNT = 3

# Game state
class Game:
    def __init__(self):

        self.grid = [['.' for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.player_pos = (0, 0)  # Starting position
        self.exit_pos = (GRID_SIZE - 1, GRID_SIZE - 1)  # Fixed exit position
        self.cabinets = []
        self.monsters = []
        self.original_monster_positions = {}  # Dictionary to store original positions
        self.swamps = []
        self.clues = []
        self.bonfires = []
        self.clues_collected = 0
        self.first_exit_visit = False
        self.inventory = []
        self.monster_chasing = [False for _ in range(NUM_MONSTERS)]
        self.game_over = False
        self.researcher = Researcher()
        self.researcher.set_character_background()
        self.researcher.update_memory(f"The researcher is in: {self.player_pos}")

        # Place monsters, swamps, and cabinets
        self.place_objects('M', NUM_MONSTERS, self.monsters)  # M for Monster
        self.place_objects('S', NUM_SWAMPS, self.swamps)  # S for Swamp
        self.place_objects('C', CABINET_LOCATIONS, self.cabinets)  # C for Cabinet

        for i, monster_pos in enumerate(self.monsters):
            self.original_monster_positions[i] = monster_pos  # Store original position

        # Set exit
        self.grid[self.exit_pos[0]][self.exit_pos[1]] = 'E'

    def place_objects(self, symbol, count, obj_list):
        placed = 0
        while placed < count:
            x = random.randint(0, GRID_SIZE - 1)
            y = random.randint(0, GRID_SIZE - 1)
            if (x, y) != self.player_pos and self.grid[x][y] == '.':
                self.grid[x][y] = symbol
                obj_list.append((x, y))
                placed += 1

    def display_grid(self):
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if (x, y) == self.player_pos:
                    print('P', end=' ')  # P for Player
                else:
                    print(self.grid[x][y], end=' ')
            print()

    def get_visible_area(self):
        visible = []
        px, py = self.player_pos
        for x in range(max(0, px - 2), min(GRID_SIZE, px + 3)):
            for y in range(max(0, py - 2), min(GRID_SIZE, py + 3)):
                if self.grid[x][y] == 'E' or (self.first_exit_visit and (x, y) in self.bonfires):
                    visible.append((x, y, self.grid[x][y]))
        return visible

    def move_player(self, dx, dy):
        x, y = self.player_pos
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < GRID_SIZE and 0 <= new_y < GRID_SIZE:
            if self.grid[new_x][new_y] == 'S':
                print("You fell into a swamp! Game over.")
                self.game_over = True
            elif self.grid[new_x][new_y] == 'M':
                print("A monster caught you! Game over.")
                self.game_over = True
            else:
                self.player_pos = (new_x, new_y)
                print(f"You moved to ({new_x}, {new_y}).")
                if (new_x, new_y) == self.exit_pos:
                    self.handle_exit()

    def handle_exit(self):
        if not self.first_exit_visit:
            print("You reached the exit, but it's locked! You need to find 3 clues.")
            self.first_exit_visit = True
            self.spawn_clue(1)
        elif self.clues_collected < CLUE_COUNT:
            print(f"You need {CLUE_COUNT - self.clues_collected} more clues to unlock the exit.")
        else:
            print("You collected all the clues and unlocked the exit! You win!")
            self.game_over = True

    def spawn_clue(self, clue_number):
        placed = False
        while not placed:
            x = random.randint(0, GRID_SIZE - 1)
            y = random.randint(0, GRID_SIZE - 1)
            if (x, y) not in self.clues and self.grid[x][y] == '.':
                self.grid[x][y] = f'C{clue_number}'
                self.clues.append((x, y))
                placed = True
        self.bonfires.append(self.exit_pos)  # Light the bonfire after visiting exit

    def collect_clue(self):
        for clue in self.clues:
            if clue == self.player_pos:
                self.clues.remove(clue)
                self.clues_collected += 1
                print(f"You found Clue {self.clues_collected}!")
                if self.clues_collected < CLUE_COUNT:
                    self.spawn_clue(self.clues_collected + 1)
                else:
                    print("All clues collected! Return to the exit.")
                return

    def move_monsters(self):
        for i, monster_pos in enumerate(self.monsters):
            if self.monster_chasing[i]:
                self.chase_player(i)
            else:
                self.return_monster(i)

    def chase_player(self, monster_index):
        mx, my = self.monsters[monster_index]
        px, py = self.player_pos
        if abs(mx - px) <= 1 and abs(my - py) <= 1:
            self.monster_chasing[monster_index] = True
            print(f"Monster {monster_index+1} is chasing you!")
            if mx < px:
                mx += 1
            elif mx > px:
                mx -= 1
            if my < py:
                my += 1
            elif my > py:
                my -= 1
            self.monsters[monster_index] = (mx, my)
            if (mx, my) == self.player_pos:
                print("A monster caught you! Game over.")
                self.game_over = True
        else:
            self.monster_chasing[monster_index] = False

    def return_monster(self, monster_index):
        mx, my = self.monsters[monster_index]
        original_pos = self.original_monster_positions[monster_index]
        ox, oy = original_pos

        if (mx, my) != original_pos:
            # Move the monster one step towards its original position
            if mx < ox:
                mx += 1
            elif mx > ox:
                mx -= 1
            if my < oy:
                my += 1
            elif my > oy:
                my -= 1
            self.monsters[monster_index] = (mx, my)
            print(f"Monster {monster_index + 1} is returning to its original position ({ox}, {oy}).")

    def hide_in_cabinet(self):
        """
        Puts the player into hiding, removing them from the grid and stopping monster chases.
        Limits the player's vision to the immediate area around the cabinet.
        """
        x, y = self.player_pos
        if self.grid[x][y] == 'C':
            print("You are hiding in a cabinet. The monsters stop chasing you.")
            for i in range(len(self.monsters)):
                self.monster_chasing[i] = False  # Stop monsters from chasing

            # Hide the player by moving them 'off-grid' temporarily
            self.player_hidden = True
            self.player_pos = (-1, -1)  # 'Invisible' position
            self.limited_vision_cabinet(x, y)
        else:
            print("You are not in a cabinet.")

    def limited_vision_cabinet(self, cx, cy):
        """
        Limits the player's vision to the area around the cabinet (1-square radius).
        """
        print("Your vision is limited while hiding in the cabinet.")
        for x in range(max(0, cx - 1), min(GRID_SIZE, cx + 2)):
            for y in range(max(0, cy - 1), min(GRID_SIZE, cy + 2)):
                print(f"({x}, {y}): {self.grid[x][y]}")

    def exit_cabinet(self):
        """
        Exits the cabinet, restoring the player to their last known position and re-enabling monster chases.
        """
        if self.player_hidden:
            print("You exit the cabinet and return to the game.")
            # Restore player position to the last cabinet position
            self.player_pos = self.last_cabinet_pos
            self.player_hidden = False  # Player is no longer hidden

            # Re-enable monster chasing
            for i in range(len(self.monsters)):
                self.monster_chasing[i] = True  # Monsters resume chasing
        else:
            print("You are not hiding in a cabinet.")

    def play_turn(self, command):
        if "move" in command:
            direction = command.split()[-1]
            if direction == "up":
                self.move_player(-1, 0)
            elif direction == "down":
                self.move_player(1, 0)
            elif direction == "left":
                self.move_player(0, -1)
            elif direction == "right":
                self.move_player(0, 1)
            self.collect_clue()
            # After each move, check if the player hides in a cabinet
            if self.grid[self.player_pos[0]][self.player_pos[1]] == 'C':
                self.hide_in_cabinet()
        elif command == "inventory":
            print("Inventory:")
            for item in self.inventory:
                print(item)
        elif command == "exit cabinet":
            self.exit_cabinet()  # Call the exit cabinet method
        else:
            print("Unknown command.")
        self.move_monsters()

    def add_to_inventory(self, item):
        """
        Add item to inventory and update memory.
        """
        if item not in self.inventory:
            self.inventory.append(item)

    def get_researcher_response(self, player_command):
        """
        Get a response from the LLM based on the player's command.
        """
        prompt = (
            f"The player said: {player_command}\n"
            "You are scared, and desperately need to escape."
            f"{self.researcher.get_full_memory()}\n"
            "What should the researcher do?\n"
            "The researcher can do the following things:\n"
            "1. move up, down, left, or right. if a cabinet exists in the location moved, automatically enter cabinet.\n"
            "2. state surroundings\n"
            "3. exit cabinet\n"
        )
        response = self.researcher.llm.invoke(prompt).content
        return response

    def player_input(self, command):
        if command == "check inventory":
            self.play_turn('inventory')
        else:
            response = self.get_researcher_response(command)
            if "move" in response:
                self.play_turn(response)
            elif "exit cabinet" in response:
                self.play_turn(response)
            else:
                print(f"The researcher said: {response}")
        self.researcher.update_memory(f"The researcher is in: {self.player_pos}")
        self.researcher.update_memory(f"The researcher sees: {self.get_visible_area()}")


In [44]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

class Researcher:
    def __init__(self):
        self.memory = ConversationBufferMemory()  # Initialize memory
        self.llm = ChatOpenAI(model="gpt-4", api_key = key, temperature=1.5)  # Initialize LLM
        self.surroundings = "You are in a dark spacecraft, covered in alien slime with tentacle monsters lurking around."
        self.visited_locations = []
        self.inventory = []

    def set_character_background(self):
        """
        Set the initial background and introduce the character.
        """
        introduction = (
            "You are a woman researcher in your early 20s, stranded in a dark spacecraft.\n"
            "You are scared, and in desperate need to escape.\n"
            "You found a walkie-talkie, and the player is your only help. "
            "Your goal is to reach the emergency exit vehicle, marked by a bonfire, "
            "but you need to gather three elements: a card key, emergency rations, and research findings."
            "You are in a 2D grid, and you can move up, down, left, right 1 square per turn."
        )
        self.memory.save_context({"background": introduction}, {"message": introduction})
        self.memory.save_context({"surroundings": self.surroundings}, {"message": self.surroundings})
        return introduction

    def update_memory(self, message):
        """
        Update memory with new surroundings or inventory changes.
        """
        self.memory.save_context({"message": message}, {"message": message})

    def add_to_inventory(self, item):
        """
        Add item to inventory and update memory.
        """
        if item not in self.inventory:
            self.inventory.append(item)
            self.update_memory(f"You added {item} to your inventory.")

    def mark_location_visited(self, location):
        """
        Mark a location as visited and update memory.
        """
        if location not in self.visited_locations:
            self.visited_locations.append(location)
            self.update_memory(f"You visited {location}.")
    def get_full_memory(self):
        """
        Retrieve the full context from memory for LLM input.
        """
        return self.memory.load_memory_variables({})['history']

In [52]:
def main():
    game = Game()
    print("Welcome to Echoes of Isolation!")
    introduction = (
            "You are a woman researcher in your early 20s, stranded in a dark spacecraft.\n"
            "You are scared, and in desperate need to escape.\n"
            "You found a walkie-talkie, and the player is your only help. "
            "Your goal is to reach the emergency exit vehicle, marked by a bonfire, "
            "but you need to gather three elements: a card key, emergency rations, and research findings."
            "You are in a 2D grid, and you can move up, down, left, right 1 square per turn."
        )
    print(game.researcher.llm.invoke(f"Player said : Where are you?.What would the researcher say?").content)
    while True:
        command = input("Enter a command: ")
        if command.lower() == "quit":
            break
        game.player_input(command)

    print("Game Over!")